In [1]:
txt_new = "Section : Dataset. Squad is a dataset on a large set of Wikipedia articles . Metrics used are : Exact Match ( EM ) and a softer metric , F1 score. Section: Model Details. Each paragraph and question are tokenized by a regular - expression - based word tokenizer ( PTB Tokenizer ) and fed into the model. Section : Results The results of our model and competing approaches on the hidden test are summarized in Table [ reference] . BiDAF ( ensemble ) achieves an EM score of 73.3 and an F1 score of 81.1 , outperforming all previous approaches "

In [2]:
txt2 = 'INTRODUCTION Generative artificial intelligence has witnessed an unprecedented growth, with applications such as ChatGPT becoming the fastest adopted consumer software in history [1]. These generative, general-purpose (language) models, characterized by their capabilities to creatively create text, audio [2], and images [3] in natural interaction, have swiftly permeated our daily lives. In the face of such rapid progress, it is essential to consider the impact of these generative artificial intelligence technologies on the future of intelligent vehicles and the transformative potential they hold for the automotive industry.'
txt2 = 'The king lost the war and his kingdom in the process'

In [3]:
txt2 = 'Introduction The rapid development of deep learning and the explosion of biological data have led to the creation of efficient models that assist clinical decision-making [1, 16, 6, 11, 25]. In recent years, medication recommendation, a fundamental yet crucial branch of clinical decision-making, has attracted significant attention from a large number of researchers [13, 20, 24, 12], which aims to recommend medications to treat the diagnosed disease of patients. Existing medication recommendation models mainly focus on digging into the representation pattern of electronic health records (EHR).'

In [4]:
keys2 = ['Introduction' ,' development',' deep',' learning', ' biological',' data', ' effecient', '  models' ,' decision-making', ' medication', ' recommendation' ,' researchers', ' patients']

In [7]:
import torch
keys=["Section", " Dataset", " Squad", " Wikipedia", " Metrics"," Exact", " Match", " F1", " score", " model", " Details", " based", " word", " tokenizer", " results", " BiDAF", " EM"]
key_tokens =[]
for key in keys2:
    tokens = tokenizer1.encode(key,return_tensors ='pt')[0]
    for i in range(len(tokens)-2):
        key_tokens.append(tokens[i+1])
# key_tokens.append(2)
key_tokens =torch.Tensor(key_tokens)


In [1]:
from transformers import AutoTokenizer, AutoModel, utils
utils.logging.set_verbosity_error()
tokenizer1 = AutoTokenizer.from_pretrained("/data1/venkat/program/master/tarun/scratch_train/ouput_pretrain_k_1")
model1 = AutoModel.from_pretrained("/data1/venkat/program/master/tarun/scratch_train/ouput_pretrain_k_1")
# txt = "The scientific document is too long to read."
# model1(tokenizer1.encode(txt_new,return_tensors ='pt'))[-1][0][0]

In [2]:
from transformers import AutoTokenizer, AutoModel, utils
utils.logging.set_verbosity_error()
tokenizer2 = AutoTokenizer.from_pretrained("/data1/venkat/program/master/tarun/scratch_train/ouput_pretrain_k_1")
model2 = AutoModel.from_pretrained("/data1/venkat/program/master/tarun/scratch_train/output_pretraining_wihtout_glob")
# txt = "The scientific document is too long to read."
# model2(tokenizer1.encode(txt_new,return_tensors ='pt'))[-1]

In [8]:
import torch
l = len(tokenizer1.encode(txt2,return_tensors ='pt')[0])
token_val = tokenizer1.encode(txt2,return_tensors ='pt')[0]
attentions1=model1(tokenizer1.encode(txt2,return_tensors ='pt'))[-1]
attentions2=model2(tokenizer1.encode(txt2,return_tensors ='pt'))[-1]
att1 = []
att2 =[] 
for i in range(12):
    a1 = torch.zeros([1,12,l,l])
    a2 = torch.zeros([1,12,l,l])
    for j in range(12):
        for k in range(l):
            a1[0][j][k] = attentions1[i][0][j][k][128-k:128+l-k]
            a2[0][j][k] = attentions2[i][0][j][k][128-k:128+l-k]
    att1.append(a1)
    att2.append(a2)
 

In [8]:
inputs = tokenizer1.encode(txt_new, return_tensors='pt')
# outputs = model(inputs)
# attention = outputs[-1]  # Output includes attention weights when output_attentions=True
tokens = tokenizer1.convert_ids_to_tokens(31673) 
tokens

'score'

In [9]:
indices = []
for keyt in key_tokens:
        

        try:
                locs=(token_val == keyt).nonzero(as_tuple=True)[0]
                for loc in locs:
                        indices.append(loc)

        except:
                print((token_val == keyt).nonzero(as_tuple=True)[0][0])

        # for loc in locs:
        #         indices.append(locs)
indices= torch.sort(torch.Tensor(indices))[0]

In [42]:
indices

tensor([ 1.,  4.,  6.,  7., 12., 13., 21., 25., 26., 27., 43., 44., 53., 54.,
        55., 66., 87., 91., 92., 93.])

In [10]:
import torch
l = len(indices)
token_val = tokenizer1.encode(txt2,return_tensors ='pt')[0]
attentions1=model1(tokenizer1.encode(txt2,return_tensors ='pt'))[-1]
attentions2=model2(tokenizer1.encode(txt2,return_tensors ='pt'))[-1]
att1 = []
att2 =[] 
for i in range(12):
    a1 = torch.zeros([1,12,l,l])
    a2 = torch.zeros([1,12,l,l])
    for j in range(12):
        for k in range(l):
            for m in range(l):
                a1[0][j][k][m] = attentions1[i][0][j][int(indices[k])][int(128+indices[m]-indices[k])]
                a2[0][j][k][m] = attentions2[i][0][j][int(indices[k])][int(128+indices[m]-indices[k])]
    att1.append(torch.mul(a1,10))
    att2.append(torch.mul(a2,10))

In [29]:
indices

tensor([ 1.,  2.,  3.,  4.,  6., 20., 21., 35., 93.])

tensor([ 2444.,  6997., 46162.,  7744., 40025.,  3693.,  2013., 37465., 29812.,
        29665.,   534., 10311., 31666., 41038.,  4528., 25017.,  2544., 42732.,
        17855., 20846.,  4255.,  1075.,  9719., 21267.,  1506.])

In [15]:
torch.mul(att2[0][0][0],10).shape

torch.Size([34, 34])

In [11]:
tokens = tokenizer1.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])

In [14]:
tokens = tokenizer1.convert_ids_to_tokens(tokenizer1.encode(txt2))

In [12]:
tokens

['Introduction',
 'Ġdevelopment',
 'Ġdeep',
 'Ġlearning',
 'Ġbiological',
 'Ġdata',
 'Ġmodels',
 'Ġdecision',
 '-',
 'making',
 'Ġmedication',
 'Ġrecommendation',
 'Ġdecision',
 '-',
 'making',
 'Ġresearchers',
 'Ġpatients',
 'Ġmedication',
 'Ġrecommendation',
 'Ġmodels']

In [48]:
att3=[]
for i in range(len(att1)):
    att3.append(att1[i]-att2[i])

In [63]:
c1=0
c2=0
c3=0
for i in range(len(att1)):
    for j in range(att1[0].shape[1]):
        c1+= att1[i][0][j][0].sum()
        c2+= att2[i][0][j][0].sum()
        c3+=20
        # for k in range(att1[0].shape[1]):
        #     for l in range(att1[0].shape[2]):
                


In [64]:
c2

tensor(142.2857, grad_fn=<AddBackward0>)

In [50]:
att1[0].shape

torch.Size([1, 12, 20, 20])

In [14]:
from bertviz import head_view
head_view(att1, tokens)

<IPython.core.display.Javascript object>

In [13]:
from bertviz import head_view
head_view(atts[5], tokss[5])

<IPython.core.display.Javascript object>

In [5]:
with open('CIKM_data/doc3.txt') as f:
    lines =f.readlines()

In [6]:
data=[]
for i in range(len(lines)):
    

    if(lines[i][0:12]=='Section name'):

        entry={}
        entry["Sec"]=lines[i][14:-1]
        entry["Para"] = lines[i+1][10:-1]
        j=0
        while(lines[i+2+j][0:10] !='Key words:'):
            j+=1
         

        entry["Keys"] = lines[i+2+j][12:-2].split(',')

        data.append(entry)
    

In [19]:
model = model1
tokenizer = tokenizer1
import torch
atts=[]
tokss=[]
sump=[]
for i1 in range(70):
         
        
    txt = str(data[i1]["Sec"]+data[i1]["Para"][0:300])
    key_tokens=[]
    key_arr = list(data[i1]["Keys"])
    
    key_arr.insert(0,str(data[i1]["Sec"]))
    for key in key_arr:
        toks = tokenizer.encode(key, return_tensors='pt')[0]
        for i in range(len(toks)-2):
            key_tokens.append(toks[i+1])
    key_tokens =torch.Tensor(key_tokens)
    indices=[]
    token_val = tokenizer.encode(txt,return_tensors='pt')[0]
    for keyt in key_tokens:
        try:
            locs=(token_val==keyt).nonzero(as_tuple=True)[0]
            for loc in locs:
                indices.append(loc)
        except:
            print((token_val==keyt).nonzero(as_tuple=True)[0][0])

    indices =torch.sort(torch.Tensor(indices))[0]

    
    import torch
    l = len(indices)
    # token_val = tokenizer.encode(txt,return_tensors='pt')[0]
    tokenizer.encode(txt,return_tensors='pt')[0]
    attentions = model(tokenizer.encode(txt,return_tensors='pt'))[-1]
    att=[]
    for i in range(12):
        a=torch.zeros([1,12,l,l])
        for j in range(12):
            for k in range(l):
                for m in range(l):  
                    try:a[0][j][k][m] = 10*attentions[i][0][j][int(indices[k])][int(128+indices[m]-indices[k])]
                    except: print(j,k,m)
        att.append(a)
    tokens = tokenizer.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])
    sum=0
    for i in range(len(att)):
        for j in range(att[0].shape[1]):
            sum+= att[i][0][j][0:len(tokenizer.encode(data[i1]["Sec"],return_tensors='pt')[0])-2].sum()
    sump.append(sum)
    # atts.append(att)
    # tokss.append(tokens)

1 9 26


In [14]:
model = model2
tokenizer = tokenizer1
import torch
atts=[]
tokss=[]
sump2=[]
for i1 in range(10):
         
        
    txt = str(data[i1]["Sec"]+data[i1]["Para"][0:300])
    key_tokens=[]
    key_arr = list(data[i1]["Keys"])
    
    key_arr.insert(0,str(data[i1]["Sec"]))
    for key in key_arr:
        toks = tokenizer.encode(key, return_tensors='pt')[0]
        for i in range(len(toks)-2):
            key_tokens.append(toks[i+1])
    key_tokens =torch.Tensor(key_tokens)
    indices=[]
    token_val = tokenizer.encode(txt,return_tensors='pt')[0]
    for keyt in key_tokens:
        try:
            locs=(token_val==keyt).nonzero(as_tuple=True)[0]
            for loc in locs:
                indices.append(loc)
        except:
            print((token_val==keyt).nonzero(as_tuple=True)[0][0])

    indices =torch.sort(torch.Tensor(indices))[0]

    
    import torch
    l = len(indices)
    # token_val = tokenizer.encode(txt,return_tensors='pt')[0]
    tokenizer.encode(txt,return_tensors='pt')[0]
    attentions = model(tokenizer.encode(txt,return_tensors='pt'))[-1]
    att=[]
    for i in range(12):
        a=torch.zeros([1,12,l,l])
        for j in range(12):
            for k in range(l):
                for m in range(l):  
                    try:a[0][j][k][m] = 10*attentions[i][0][j][int(indices[k])][int(128+indices[m]-indices[k])]
                    except: print(j,k,m)
        att.append(a)
    tokens = tokenizer.convert_ids_to_tokens(token_val[[list(indices.type(torch.int))]])
    sum=0
    for i in range(len(att)):
        for j in range(att[0].shape[1]):
            sum+= att[i][0][j][0:len(tokenizer.encode(data[i1]["Sec"],return_tensors='pt')[0])-2].sum()
    sump2.append(sum)
    atts.append(att)
    tokss.append(tokens)

In [16]:
for i in range(len(sump)):
    print(100*(sump[i]-sump2[i])/sump2[i])

tensor(15.1721, grad_fn=<DivBackward0>)
tensor(13.3302, grad_fn=<DivBackward0>)
tensor(9.1370, grad_fn=<DivBackward0>)
tensor(14.4857, grad_fn=<DivBackward0>)
tensor(11.5627, grad_fn=<DivBackward0>)
tensor(8.5933, grad_fn=<DivBackward0>)
tensor(27.4503, grad_fn=<DivBackward0>)
tensor(-2.4493, grad_fn=<DivBackward0>)
tensor(21.6101, grad_fn=<DivBackward0>)
tensor(7.5782, grad_fn=<DivBackward0>)
